<!--BOOK_INFORMATION-->
<img align="left" style="width:80px;height:98px;padding-right:20px;" src="https://raw.githubusercontent.com/joe-papa/pytorch-book/main/files/pytorch-book-cover.jpg">

This notebook contains an excerpt from the [PyTorch Pocket Reference](http://pytorchbook.com) book by [Joe Papa](http://joepapa.ai); content is available [on GitHub](https://github.com/joe-papa/pytorch-book).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/joe-papa/pytorch-book/blob/main/07_01_Deploying_PyTorch_to_Production.ipynb)

# Chapter 7 - Deploy to Production

We have omitted code for this section due since it's not intended for notebooks. 

We hope to provide links to additional repositories in the future.

In [ ]:
from torchvision.models import vgg16

model = vgg16(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


In [ ]:
import numpy as np

model_parameters = filter(lambda p: 
      p.requires_grad, model.parameters())

params = sum([np.prod(p.size()) for 
      p in model_parameters])
print(params)

# out: 138357544

138357544


In [ ]:
import torch 

model = vgg16(pretrained=True)
example_input = torch.rand(1, 3, 224, 224)
torchscript_model = torch.jit.trace(model, 
                            example_input)
torchscript_model.save("traced_vgg16_model.pt")

/usr/local/lib/python3.7/dist-packages/torch/jit/_trace.py:966: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
With rtol=1e-05 and atol=1e-05, found 1000 element(s) (out of 1000) whose difference(s) exceeded the margin of error (including 0 nan comparisons). The greatest difference was 2.8998576402664185 (-1.6628050804138184 vs. 1.2370525598526), which occurred at index (0, 2).
  _module_class,


In [ ]:
import torch.nn as nn

class ControlFlowModel(nn.Module):
  def __init__(self, N):
    super(ControlFlowModel, self).__init__()
    self.fc = nn.Linear(N,100)
 
  def forward(self, input):
    if input.sum() > 0:
      output = input
    else:
      output = -input
    return output

model = ControlFlowModel(10)
torchcript_model = torch.jit.script(model)
torchscript_model.save("scripted_vgg16_model.pt")

In [ ]:
model = vgg16(pretrained=True)
example_input = torch.rand(1, 3, 224, 224)
onnx_model = torch.onnx.export(model, 
                               example_input, 
                               "vgg16.onnx")

In [ ]:
!pip install onnx

     |████████████████████████████████| 14.5MB 317kB/s 


In [ ]:
import onnx

model = onnx.load("vgg16.onnx")
onnx.checker.check_model(model)
onnx.helper.printable_graph(model.graph)

'graph torch-jit-export (\n  %input.1[FLOAT, 1x3x224x224]\n) initializers (\n  %features.0.weight[FLOAT, 64x3x3x3]\n  %features.0.bias[FLOAT, 64]\n  %features.2.weight[FLOAT, 64x64x3x3]\n  %features.2.bias[FLOAT, 64]\n  %features.5.weight[FLOAT, 128x64x3x3]\n  %features.5.bias[FLOAT, 128]\n  %features.7.weight[FLOAT, 128x128x3x3]\n  %features.7.bias[FLOAT, 128]\n  %features.10.weight[FLOAT, 256x128x3x3]\n  %features.10.bias[FLOAT, 256]\n  %features.12.weight[FLOAT, 256x256x3x3]\n  %features.12.bias[FLOAT, 256]\n  %features.14.weight[FLOAT, 256x256x3x3]\n  %features.14.bias[FLOAT, 256]\n  %features.17.weight[FLOAT, 512x256x3x3]\n  %features.17.bias[FLOAT, 512]\n  %features.19.weight[FLOAT, 512x512x3x3]\n  %features.19.bias[FLOAT, 512]\n  %features.21.weight[FLOAT, 512x512x3x3]\n  %features.21.bias[FLOAT, 512]\n  %features.24.weight[FLOAT, 512x512x3x3]\n  %features.24.bias[FLOAT, 512]\n  %features.26.weight[FLOAT, 512x512x3x3]\n  %features.26.bias[FLOAT, 512]\n  %features.28.weight[FLOAT